In [ ]:
import DatasetPrepare
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor , AdaBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [ ]:
#Load data and create new column
df = pd.DataFrame(DatasetPrepare.getFujiDatasets())
df['y'] = df['Fuji'].shift(-1)
df = df.dropna(subset=['y'])
df['baseline_pred'] = df['Fuji']

In [ ]:
X_train = df['Fuji'].values.reshape(-1,1)
y_train = df['y'].values.reshape(-1,1)
X_test = df['Fuji'].values.reshape(-1,1)

In [ ]:
#Initialize Random Forest Regressor Model
random_reg = RandomForestRegressor()

#Fit the model
random_reg.fit(X_train, y_train)
#Make predictions
predictions_random = random_reg.predict(X_test)

#Assign predictions to new column
df['random_pred'] = predictions_random

random_mae = mean_absolute_error(y_train, predictions_random)
random_mse = mean_squared_error(y_train, predictions_random)
random_r2 = r2_score(y_train, predictions_random)
print("MAE:", random_mae)
print("MSE:", random_mse)
print("R-squared:", random_r2)

In [ ]:
#Initialize Gradient Boosting Regressor Model
gradient_reg = GradientBoostingRegressor()

#Fit the model
gradient_reg.fit(X_train, y_train)
#Make predictions
predictions_gradient = gradient_reg.predict(X_test)

#Assign predictions to new column
df['gradient_pred'] = predictions_gradient

#Evaluate
gradient_mae = mean_absolute_error(y_train, predictions_gradient)
gradient_mse = mean_squared_error(y_train, predictions_gradient)
gradient_r2 = r2_score(y_train, predictions_gradient)
print("MAE:", gradient_mae)
print("MSE:", gradient_mae)
print("R-squared:", gradient_r2)

In [ ]:
#Initialize Ada Boost Regressor Model
ada_reg = AdaBoostRegressor()

#Fit the model
ada_reg.fit(X_train, y_train)
#Make predictions
predictions_ada = ada_reg.predict(X_test)

#Assign predictions to new column
df['ada_pred'] = predictions_ada

#Evaluate
ada_mae = mean_absolute_error(y_train, predictions_ada)
ada_mse = mean_squared_error(y_train, predictions_ada)
ada_r2 = r2_score(y_train, predictions_ada)
print("MAE:", ada_mae)
print("MSE:", ada_mse)
print("R-squared:", ada_r2)

In [ ]:
# evaluting
f = plt.figure()
f.set_figwidth(40)
f.set_figheight(20)

plt.plot(df.index, df['baseline_pred'], label='baseline')
plt.plot(df.index, df['random_pred'], label='Random Forest')
plt.plot(df.index, df['gradient_pred'], label='Gradient Boosting')
plt.plot(df.index, df['ada_pred'], label='Ada Boost')
plt.title('Prediction')
plt.legend()
plt.show()

In [ ]:
#Implement Mape Function
def mape(y_true, y_pred):
    return round(np.mean(np.abs((y_true - y_pred) / y_true)) * 100, 2)
  
random_mape = mape(y_train, predictions_random)
gradient_mape = mape(y_train, predictions_gradient)
ada_mape = mape(y_train, predictions_ada)

x = np.array(["Random Forest", "Gradient Boosting", "Ada Boost"])
y = np.array([random_mape, gradient_mape, ada_mape])

plt.bar(x,y)
plt.title('MAPE')
plt.legend()
plt.show()


In [ ]:
x = np.array(["Random Forest", "Gradient Boosting", "Ada Boost"])
y = np.array([random_r2, gradient_r2, ada_r2])

plt.bar(x,y)
plt.title('R2 Score')
plt.legend()
plt.show()